In [ ]:
# Importamos JSON
import json

# Guardamos las credenciales para consumir api de twitter
credentials = {
    "KEY": "c0Jlf7B9Ez56i6rsnBSeiBCii",
    "KEY_SECRET": "yuCAV2TdGrvitWBGQondppKaxNlQfnrc5krRCYvNpu3miY2gQQ",
    "TOKEN": "1279067386520440833-YLZvyZzPDZlQ3bGa5SH2fhugVPl4Cs",
    "ACCESS_SECRET": "DZuy6rzCJFZ4723jVl2gITgf2jCXr1cqlFRtQXFytrIKD"
}

credentials_PRO = {
    "KEY": "XqNe5tbaycDxrrIOVqHR1NoPk",
    "KEY_SECRET": "wahClTvDgP1Kb7l8QgHJD6DTFhhgY9usDfFaXSY5bnLdlrnv4q",
    "TOKEN": "1279025620287205377-OlltfTVwLqHRXklJPGAlAWvF7gC0bQ",
    "ACCESS_SECRET": "rJX1qRcToTyZq4Aqam18YjT1rbmFRhKvNflgB5RELz9aM"
}

credentials
credentials_PRO

In [ ]:
# Intslamos Twythom
!pip install twython

In [ ]:
# Importar TwyThon (Consumir api de twitter)
from twython import Twython

# Objeto crendeiales
tweets = Twython(credentials["KEY"], credentials["KEY_SECRET"])

# Palabra a consultar
word = "Colombia"
# Consulta
query = {
    'q': word,
    'result_type': 'popular',
    'count': 2,
    'lang': 'es'
}

query

In [ ]:
consulta = tweets.search(**query)['statuses']
#consulta

In [ ]:
import pandas as pd

df = pd.json_normalize(consulta)
df

In [ ]:
## consultar api tiempo real
from twython import TwythonStreamer

## Se guardara en un archivo .csv
import csv

#procesar los datos con filtro
def process_tweet(tweet):
    # guardar hashtag / contenido / usuario / localizacion / edad / mirar mas
    d = {}
    d["hashtags"] = [hashtag["text"] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_location'] = tweet['user']['location']
    return d

# Heredando clase
#se le pasa las credenciales (probar siempre con credentials normales.)
class Streamer(TwythonStreamer):

    #Datos recibidos
    def on_success(self, data):
        tweet_data = process_tweet(data)
        self.save_tweet(tweet_data)

    #Error
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    #Guardar en un archivo el resultados
    def save_tweet(self, tweet):
        #reempalzar si no esta trabajando en visualstudio o no esta en la carpeta origen del proyecto
        #with open(r'tweets.csv', 'a', encoding="utf-8") as file: 
        with open(r'work/slides/data/tweets.csv', 'a', encoding="utf-8") as file:
            write = csv.writer(file)
            write.writerow(list(tweet.values()))

In [ ]:
# Intanciar
stream =  Streamer(credentials['KEY'], credentials['KEY_SECRET'], credentials['TOKEN'], credentials['ACCESS_SECRET'])

#Iniciar consulta en tiempo real
stream.statuses.filter(track="Colombia")
# Para poder para la consulta y luego ver los resultados debe parar el proceso manualmente, es decir interrumpirlo.
# dando click en el boton de interrupt IPython kernel

In [ ]:
# #reempalzar si no esta trabajando en visualstudio o no esta en la carpeta origen del proyecto
# tweetsRealTime = pd.read_csv("tweets.csv")
tweetsRealTime = pd.read_csv("work/slides/data/tweets.csv")
tweetsRealTime.head()

In [ ]:
#Mostrando todo
tweetsRealTime

In [ ]:
from collections import Counter
import ast

tweets = pd.read_csv("work/slides/data/tweets.csv")
tweets

In [ ]:
# Instalamos geopy
!pip install geopy

In [ ]:
# Instalamos gmplot
!pip install gmplot

In [ ]:
df = pd.DataFrame(tweets)

df

In [ ]:
from geopy.geocoders import Nominatim
import gmplot

geolocator = Nominatim(user_agent="BD-Project-JJ")

no_location = 0

#revisar todos los tweets y agregar ubicaciones 
coordinates =  {'latitude': [], 'longitude': []}
for count, user_location in enumerate(tweets.location):
    try: 
        location = geolocator.geocode(user_location)

        #si se encuentra coordenadas 
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)

        else:
            no_location += 1
            coordinates['latitude'].append(0)
            coordinates['longitude'].append(0)

    except:
        #si hay demasiads solicitudes de conexion
        pass



In [ ]:
#coordinates['latitude']
no_location
#tweets

In [ ]:
data = tweets

data['latitude'] = coordinates['latitude']
data['longitude'] = coordinates['longitude']

data.to_csv('work/slides/data/tweetCoordinates.csv', index = False, header=True)

data


In [ ]:
# instancia
gmap = gmplot.GoogleMapPlotter(30,0,3)

#insertar puntos en el mapa pasando una lista de coordenadas
gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)

#Guardar el mapa
gmap.draw('work/slides/heatmap.html')